Vamos a listar las "jotas", es decir, el índice de las coordenadas totales, siguiendo este orden: de la esquina inferior [1,1], ese será j=1, luego j=2 corresponderá a [1,2], j=64 será [1,64] y luego j=65 será [2,1]. Recuerda que esto está ordenado como MATRICES, entonces j=5, [1,5]  es el lugar coordenada x=5 y y=1. Respeta el orden y todo saldrá bien.

In [1]:
ConjuntoDeCoordenadasTotal=Array[]

0-element Array{Array{T,N},1}

In [2]:
for j=1:64,k=1:64
    push!(ConjuntoDeCoordenadasTotal,[j,k])
end
ConjuntoDeCoordenadasTotal[65]

2-element Array{Int64,1}:
 2
 1

Por cierto, estoy SEGURO que esta NO es la forma más inteligente de hacerlo, pero funciona. Ya después optimizamos.

In [3]:
ConjuntoDeCoordenadasTotal[65]

2-element Array{Int64,1}:
 2
 1

In [4]:
BceroDura=readdlm("BceroDura.dat");

In [5]:
xk=ConjuntoDeCoordenadasTotal[70]

2-element Array{Int64,1}:
 2
 6

In [6]:
BceroDura[64,64]=137.089 #Correccion por convergencia, hay que checar el notebook anterior y ver porque pasa eso...

137.089

In [7]:
xk=ConjuntoDeCoordenadasTotal[70]
BceroDura[xk...], BceroDura[2,6]


(7.700687690958998,7.700687690958998)

In [8]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

Esta parte de abajo debe ser trivialmente paralelizable, ¿no?

In [9]:
jmax=128
xconstante=[64,64]
Kduras=zeros(jmax,jmax);
@time for j=1:jmax
     xj=ConjuntoDeCoordenadasTotal[j]
     for k=1:j
            xk=ConjuntoDeCoordenadasTotal[k]
        for l=1:jmax
            xl=ConjuntoDeCoordenadasTotal[l]
            Kduras[j,k]+=BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
            end
    end
end

  3.006448 seconds (25.39 M allocations: 920.242 MB, 2.95% gc time)


In [ ]:
jmax=4096
KdurasParalel=SharedArray(Float64, jmax,jmax)

@time @sync @parallel for j=1:jmax
     xj=ConjuntoDeCoordenadasTotal[j]
     for k=1:j
            xk=ConjuntoDeCoordenadasTotal[k]

         for l=1:jmax
            xl=ConjuntoDeCoordenadasTotal[l]
            KdurasParalel[j,k]+= BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
         
       
        end
  #  println("hola loco ", KdurasParalel[j,k])
    end
end 

In [ ]:
writedlm("KDurasParalel.dat", KdurasParalel)

In [ ]:
KdurasParalel[22,20] -Kduras[22,20]

In [ ]:
using PyPlot

In [ ]:
ion()
figuru=imshow(Kduras, origin="upper",interpolation="nearest",cmap="viridis") 
cb=colorbar(figuru)

Hay algo raro... me está saliendo Ciclica la integral... ¿tiene sentido?

In [ ]:
Kduras[1:64,1:64]-Kduras[65:128,65:128]

In [ ]:
extrema(Kduras)